In [1]:
import os
import time
import shutil
import numpy as np
import tensorflow as tf
import core.utils as utils
from tqdm import tqdm
from core.dataset import Dataset
from core.yolov3 import YOLOV3
from core.config import cfg
import xml.etree.ElementTree as ET
from PIL import Image



f2 = open('../train_ann.txt','r')
temp = f2.readlines()
def classToInt(className):
    #cause the class name is recorded as string, so have to translate to integer
    if className == 'aquarium':
        return 0
    if className == 'bottle':
        return 1
    if className == 'bowl':
        return 2
    if className == 'box':
        return 3
    if className == 'bucket':
        return 4
    if className == 'plastic_bag':
        return 5
    if className == 'plate':
        return 6
    if className == 'styrofoam':
        return 7
    if className == 'tire':
        return 8
    if className == 'toilet':
        return 9
    if className == 'tub':
        return 10
    if className == 'washing_machine':
        return 11
    if className == 'water_tower':
        return 12

def calcuText(xmlFile):
    #the training data has an annotation xml file
    #the file records image name,size,object location
    #this function would parses the file to tensorflow record
    
    tree = ET.parse(xmlFile)
    root = tree.getroot()
    imageName = root.find('filename').text
    image = Image.open('../aidea/train_cdc/train_images/' + imageName)
    imageBytes = image.tobytes()
    size = root.find('size')
    width=int(size[0].text)
    height=int(size[1].text)
    depth=int(size[2].text)
    features = {}
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    classes = []
    classesName = []
    temp = ""
    if(len(root.findall('object')) == 0):
        print(xmlFile)
    for i in root.findall('object'):
            temp +=  " " + str(i[1][0].text) + "," + str(i[1][1].text) + "," + str(i[1][2].text) + "," + str(i[1][3].text) + "," + str(classToInt(i[0].text))
    #0 xxx/xxx/a.jpg 453 369 473 391 1 588 245 608 268 2
    result = '../aidea/train_cdc/train_images/' + imageName + " " + temp + "\n"
    return result
import random
'''
trainTxt = open("trainval.txt",'w')
valTxt = open("val.txt",'w')
for j in range(len(temp)):
    rand = random.randint(0,100)
    example = calcuText('.' + temp[j].rstrip())
    if(rand >= 90):
        valTxt.write(example)
    else:
        trainTxt.write(example)
trainTxt.close()
valTxt.close()'''

'\ntrainTxt = open("trainval.txt",\'w\')\nvalTxt = open("val.txt",\'w\')\nfor j in range(len(temp)):\n    rand = random.randint(0,100)\n    example = calcuText(\'.\' + temp[j].rstrip())\n    if(rand >= 90):\n        valTxt.write(example)\n    else:\n        trainTxt.write(example)\ntrainTxt.close()\nvalTxt.close()'

In [2]:
#! /usr/bin/env python
# coding=utf-8
#================================================================
#   Copyright (C) 2019 * Ltd. All rights reserved.
#
#   Editor      : VIM
#   File name   : train.py
#   Author      : YunYang1994
#   Created date: 2019-02-28 17:50:26
#   Description :
#
#================================================================




class YoloTrain(object):
    def __init__(self):
        
        self.anchor_per_scale    = cfg.YOLO.ANCHOR_PER_SCALE
        self.classes             = utils.read_class_names(cfg.YOLO.CLASSES)
        self.num_classes         = len(self.classes)
        self.learn_rate_init     = cfg.TRAIN.LEARN_RATE_INIT
        self.learn_rate_end      = cfg.TRAIN.LEARN_RATE_END
        self.first_stage_epochs  = cfg.TRAIN.FISRT_STAGE_EPOCHS
        self.second_stage_epochs = cfg.TRAIN.SECOND_STAGE_EPOCHS
        self.warmup_periods      = cfg.TRAIN.WARMUP_EPOCHS
        self.initial_weight      = cfg.TRAIN.INITIAL_WEIGHT
        self.time                = time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime(time.time()))
        self.moving_ave_decay    = cfg.YOLO.MOVING_AVE_DECAY
        self.max_bbox_per_scale  = 150
        self.train_logdir        = "./data/log/train"
        self.trainset            = Dataset('train')
        self.testset             = Dataset('test')
        self.steps_per_period    = len(self.trainset)
        self.sess                = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))

        with tf.name_scope('define_input'):
            self.input_data   = tf.placeholder(dtype=tf.float32, name='input_data')
            self.label_sbbox  = tf.placeholder(dtype=tf.float32, name='label_sbbox')
            self.label_mbbox  = tf.placeholder(dtype=tf.float32, name='label_mbbox')
            self.label_lbbox  = tf.placeholder(dtype=tf.float32, name='label_lbbox')
            self.true_sbboxes = tf.placeholder(dtype=tf.float32, name='sbboxes')
            self.true_mbboxes = tf.placeholder(dtype=tf.float32, name='mbboxes')
            self.true_lbboxes = tf.placeholder(dtype=tf.float32, name='lbboxes')
            self.trainable     = tf.placeholder(dtype=tf.bool, name='training')

        with tf.name_scope("define_loss"):
            self.model = YOLOV3(self.input_data, self.trainable)
            self.net_var = tf.global_variables()
            self.giou_loss, self.conf_loss, self.prob_loss = self.model.compute_loss(
                                                    self.label_sbbox,  self.label_mbbox,  self.label_lbbox,
                                                    self.true_sbboxes, self.true_mbboxes, self.true_lbboxes)
            self.loss = self.giou_loss + self.conf_loss + self.prob_loss

        with tf.name_scope('learn_rate'):
            self.global_step = tf.Variable(1.0, dtype=tf.float64, trainable=False, name='global_step')
            warmup_steps = tf.constant(self.warmup_periods * self.steps_per_period,
                                        dtype=tf.float64, name='warmup_steps')
            train_steps = tf.constant( (self.first_stage_epochs + self.second_stage_epochs)* self.steps_per_period,
                                        dtype=tf.float64, name='train_steps')
            self.learn_rate = tf.cond(
                pred=self.global_step < warmup_steps,
                true_fn=lambda: self.global_step / warmup_steps * self.learn_rate_init,
                false_fn=lambda: self.learn_rate_end + 0.5 * (self.learn_rate_init - self.learn_rate_end) *
                                    (1 + tf.cos(
                                        (self.global_step - warmup_steps) / (train_steps - warmup_steps) * np.pi))
            )
            global_step_update = tf.assign_add(self.global_step, 1.0)

        with tf.name_scope("define_weight_decay"):
            moving_ave = tf.train.ExponentialMovingAverage(self.moving_ave_decay).apply(tf.trainable_variables())

        with tf.name_scope("define_first_stage_train"):
            self.first_stage_trainable_var_list = []
            for var in tf.trainable_variables():
                var_name = var.op.name
                var_name_mess = str(var_name).split('/')
                if var_name_mess[0] in ['conv_sbbox', 'conv_mbbox', 'conv_lbbox']:
                    self.first_stage_trainable_var_list.append(var)

            first_stage_optimizer = tf.train.AdagradOptimizer(self.learn_rate).minimize(self.loss,
                                                      var_list=self.first_stage_trainable_var_list)
            with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                with tf.control_dependencies([first_stage_optimizer, global_step_update]):
                    with tf.control_dependencies([moving_ave]):
                        self.train_op_with_frozen_variables = tf.no_op()

        with tf.name_scope("define_second_stage_train"):
            second_stage_trainable_var_list = tf.trainable_variables()
            second_stage_optimizer = tf.train.AdagradOptimizer(self.learn_rate).minimize(self.loss,
                                                      var_list=second_stage_trainable_var_list)

            with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                with tf.control_dependencies([second_stage_optimizer, global_step_update]):
                    with tf.control_dependencies([moving_ave]):
                        self.train_op_with_all_variables = tf.no_op()

        with tf.name_scope('loader_and_saver'):
            self.loader = tf.train.Saver(self.net_var)
            self.saver  = tf.train.Saver(tf.global_variables(), max_to_keep=1000)

        with tf.name_scope('summary'):
            tf.summary.scalar("learn_rate",      self.learn_rate)
            tf.summary.scalar("giou_loss",  self.giou_loss)
            tf.summary.scalar("conf_loss",  self.conf_loss)
            tf.summary.scalar("prob_loss",  self.prob_loss)
            tf.summary.scalar("total_loss", self.loss)

            logdir = "./data/log/"
            if os.path.exists(logdir): shutil.rmtree(logdir)
            os.mkdir(logdir)
            self.write_op = tf.summary.merge_all()
            self.summary_writer  = tf.summary.FileWriter(logdir, graph=self.sess.graph)


    def train(self):
        self.sess.run(tf.global_variables_initializer())
        try:
            print('=> Restoring weights from: %s ... ' % self.initial_weight)
            self.loader.restore(self.sess, self.initial_weight)
        except:
            print('=> %s does not exist !!!' % self.initial_weight)
            print('=> Now it starts to train YOLOV3 from scratch ...')
            self.first_stage_epochs = 0

        for epoch in range(1, 1+self.first_stage_epochs+self.second_stage_epochs):
            if epoch <= self.first_stage_epochs:
                train_op = self.train_op_with_frozen_variables
            else:
                train_op = self.train_op_with_all_variables

            pbar = tqdm(self.trainset)
            train_epoch_loss, test_epoch_loss = [], []

            for train_data in pbar:
                _, summary, train_step_loss, global_step_val = self.sess.run(
                    [train_op, self.write_op, self.loss, self.global_step],feed_dict={
                                                self.input_data:   train_data[0],
                                                self.label_sbbox:  train_data[1],
                                                self.label_mbbox:  train_data[2],
                                                self.label_lbbox:  train_data[3],
                                                self.true_sbboxes: train_data[4],
                                                self.true_mbboxes: train_data[5],
                                                self.true_lbboxes: train_data[6],
                                                self.trainable:    True,
                })

                train_epoch_loss.append(train_step_loss)
                self.summary_writer.add_summary(summary, global_step_val)
                pbar.set_description("train loss: %.2f" %train_step_loss)

            for test_data in self.testset:
                test_step_loss = self.sess.run( self.loss, feed_dict={
                                                self.input_data:   test_data[0],
                                                self.label_sbbox:  test_data[1],
                                                self.label_mbbox:  test_data[2],
                                                self.label_lbbox:  test_data[3],
                                                self.true_sbboxes: test_data[4],
                                                self.true_mbboxes: test_data[5],
                                                self.true_lbboxes: test_data[6],
                                                self.trainable:    False,
                })

                test_epoch_loss.append(test_step_loss)

            train_epoch_loss, test_epoch_loss = np.mean(train_epoch_loss), np.mean(test_epoch_loss)
            ckpt_file = "./checkpoint/yolov3_test_loss=%.4f.ckpt" % test_epoch_loss
            log_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
            print("=> Epoch: %2d Time: %s Train loss: %.2f Test loss: %.2f Saving %s ..."
                            %(epoch, log_time, train_epoch_loss, test_epoch_loss, ckpt_file))
            self.saver.save(self.sess, ckpt_file, global_step=epoch)


os.environ["CUDA_VISIBLE_DEVICES"] = "0"
if __name__ == '__main__': YoloTrain().train()






Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use tf.cast instead.
=> Restoring weights from: ./checkpoint/yolov3_test_loss=19.5595.ckpt-228 ... 
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./checkpoint/yolov3_test_loss=19.5595.ckpt-228


train loss: 2.84: 100%|██████████| 396/396 [01:05<00:00,  7.35it/s] 


=> Epoch:  1 Time: 2019-06-20 12:40:47 Train loss: 2.30 Test loss: 19.37 Saving ./checkpoint/yolov3_test_loss=19.3706.ckpt ...


train loss: 2.87: 100%|██████████| 396/396 [00:59<00:00,  7.70it/s] 


=> Epoch:  2 Time: 2019-06-20 12:41:55 Train loss: 2.24 Test loss: 19.76 Saving ./checkpoint/yolov3_test_loss=19.7649.ckpt ...


...

=> Epoch: 340 Time: 2019-06-21 02:58:59 Train loss: 1.65 Test loss: 22.42 Saving ./checkpoint/yolov3_test_loss=22.4246.ckpt ...


train loss: 2.32: 100%|██████████| 396/396 [02:26<00:00,  3.15it/s]


=> Epoch: 341 Time: 2019-06-21 03:01:35 Train loss: 1.67 Test loss: 22.12 Saving ./checkpoint/yolov3_test_loss=22.1229.ckpt ...


train loss: 2.46:  51%|█████     | 201/396 [01:14<01:13,  2.67it/s]

KeyboardInterrupt: 